In [1]:
import random
import string

def generate_referral_code(length=6):
    """Generate a random referral code."""
    letters_and_digits = string.ascii_letters + string.digits
    return ''.join(random.choice(letters_and_digits) for _ in range(length))

def generate_unique_referral_codes(count=1):
    """Generate a specified number of unique referral codes."""
    referral_codes = set()
    while len(referral_codes) < count:
        referral_codes.add(generate_referral_code())
    return list(referral_codes)

def save_referral_codes_to_file(referral_codes, filename='code.txt'):
    """Save referral codes to a file."""
    with open(filename, 'w') as file:
        for code in referral_codes:
            file.write(code + '\n')

def main():
    
    unique_referral_codes = generate_unique_referral_codes()

   
    print("Generated Referral Codes:")
    for code in unique_referral_codes:
        print(code)

    
    save_referral_codes_to_file(unique_referral_codes)
    print(f"\nReferral codes saved to 'referral_codes.txt' file.")

if __name__ == "__main__":
    main()


Generated Referral Codes:
zB528V

Referral codes saved to 'referral_codes.txt' file.


In [2]:
import csv

def assign_codes_to_emails(email_file, code_file, output_csv):
    with open(email_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        header = reader.fieldnames
        email_column = [col for col in header if 'email' in col.lower()][0]
        name_column = [col for col in header if 'name' in col.lower()][0]
        data = list(reader)

    with open(code_file, 'r') as code_file:
        codes = code_file.read().splitlines()

    print(f"Number of emails: {len(data)}")
    print(f"Number of codes: {len(codes)}")

    if len(data) != len(codes):
        raise ValueError("Number of emails and codes do not match")

    email_code_pairs = [(row[name_column], row[email_column], code) for row, code in zip(data, codes)]

    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Name', 'Email', 'Code']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for name, email, code in email_code_pairs:
            writer.writerow({'Name': name, 'Email': email, 'Code': code})

if __name__ == "__main__":
    assign_codes_to_emails('email.csv', 'code.txt', 'output.csv')


Number of emails: 1
Number of codes: 1


In [3]:
import csv
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


def send_email(sender_email, sender_password, receiver_email, receiver_name, subject, message):
    try:
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = subject
        html_message = f"""
<html>
    <head>
        <style>
            body {{
                font-family: 'Roboto', sans-serif;
                margin: 0;
                padding: 0;
                background-color: #FF4500;
                color: #FFFFFF;
            }}
            .header {{
                background-color: #FFFFFF; 
                padding: 20px;
                text-align: center;
            }}
            .logo-text {{
                font-size: 20px;
                font-weight: bold;
                color: #6f1430; 
            }}
          
            .email-body {{
                font-size: 16px;
                line-height: 1.6;
                padding: 20px;
            }}
            .footer {{
                margin-top: 20px;
                text-align: center;
                color: #555;
            }}
            .bold-text {{
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <div class="header">
            <p class="logo-text">Hindi Club VIT Bhopal</p>
        </div>

        <div class="email-body">
            <p style='color: #555; font-size: 16px; font-weight: bold;'> Dear {receiver_name},</p>
            
            <p>Use this referral code and encourage others to register.<br> The person with the maximum number of registrations will definitely get a special perk. <br> All the best!</p>

            <p class="bold-text">{message}</p>
        </div>

        <div class="footer">
            &copy; 2024 Hindi Club. All rights reserved.
        </div>
    </body>
</html>"""



        msg.attach(MIMEText(html_message, 'html'))

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())

        print(f"Email sent to {receiver_email} with subject: {subject}")
    except Exception as e:
        print(f"Error sending email to {receiver_email}: {e}")


def send_emails(email_file, sender_email, sender_password):
    with open(email_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            receiver_email = row['Email']
            receiver_name = row['Name']
            subject = f"{receiver_name}, Your Hindi Club Referral Code"
            message = f"Your assigned referral code is: <span style='color: #555; font-size: 16px; font-weight: bold; border: 1px solid #888; padding: 5px;'>{row['Code']}</span> <br> Thank you for being a part of Hindi Club! <br><span style='color: #555; font-size: 16px; font-weight: bold;'> यत् भावो – तत् भवति</span>"


            send_email(sender_email, sender_password,
                       receiver_email, receiver_name, subject, message)


if __name__ == "__main__":
    sender_email = 'email'
    sender_password = 'pass'
    send_emails('output.csv', sender_email, sender_password)

Email sent to mausam.mishra2000@gmail.com with subject: mausam, Your Hindi Club Referral Code
